In [ ]:
from pydbgen import pydbgen
import os
import pandas as pd
from faker import Faker
fake = Faker()
import random
import numpy as np
import string
import os
from tqdm import tqdm
def randomString(stringLength):
    """Generate a random string with the combination of lowercase and uppercase letters """
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(stringLength))

# Constants

In [ ]:
DATA_SIZE = 1000000
START_DATE = '2019-09-15'
END_DATE = '2019-09-30'

In [ ]:
def randomDir():
    randint = random.randint(0,64)
    if randint > 31:
        drive_letters = ['C:', 'D:', 'H:','/', 'Z:']
        file_extensions = ['.a','.asm','.asp','.awk','.bat','.bmp','.btm','.BTM','.c','.class','.cmd','.CPP','.csv','.cur','.cxx','.CXX','.db','.def','.DES','.dlg','.dll','.don','.dpc','.dpj','.dtd','.dump','.dxp','.eng','.exe','.flt','.fmt','.font','.fp','.ft','.gif','.h','.H','.hdb','.hdl','.hid','.hpp','.hrc','.HRC','.html','.hxx','.Hxx','.HXX','.ico','.idl','.IDL','.ih','.ilb','.inc','.inf','.ini','.inl','.ins','.java','.jar','.jnl','.jpg','.js','.jsp','.kdelnk','.l','.lgt','.lib','.lin','.ll','.LN3','.lng','.lnk','.lnx','.LOG','.lst','.lst','.mac','.MacOS','.map','.mk','.MK','.mod','.NT2','.o','.obj','.par','.pfa','.pfb','.pl','.PL','.plc','.pld','.PLD','.plf','.pm','.pmk','.pre','.PRJ','.prt','.PS','.ptr','.r','.rc','.rdb','.res','.s','.S','.sbl','.scp','.scr','.sda','.sdb','.sdc','.sdd','.sdg','.sdm','.sds','.sdv','.sdw','.sdi','.seg','.SEG','.Set','.sgl','.sh','.sid','.smf','.sms','.so','.sob','.sob','.soc','.sod','.soe','.sog','.soh','.src','.srs','.SSLeay','.Static','.tab','.TFM','.thm','.tpt','.tsc','.ttf','.TTF','.txt','.TXT','.unx','.UNX','.urd','.url','.VMS','.vor','.W32','.wav','.wmf','.xml','.xpm','.xrb','.y','.yxx','.zip',]
        path_parts = fake.text(max_nb_chars=randint).split()
        path_parts = [word.replace('.','').lower() for word in path_parts]
        return os.path.join(random.choice(drive_letters), *path_parts[:-1], path_parts[-1] + random.choice(file_extensions))
    else:
        return None

In [ ]:
my_db = pydbgen.pydb()

In [ ]:
my_db.gen_data_series(data_type='date')

In [ ]:
df_employees = pd.read_csv('../airflow/data/employees.csv')

In [ ]:
df_employees.head()

In [ ]:
df_employees = my_db.gen_dataframe(50000, fields=['name','email', 'phone'], real_email=False)

In [ ]:
df_employees['address'] = pd.Series([fake.address() for i in tqdm(range(df_employees.shape[0]))])

In [ ]:
df_employees.head()

In [ ]:
df_employees.drop_duplicates(subset=['email']).to_csv('employees.csv')

# Now work on the synthetic email data

In [ ]:
column_names = ["guid"#: "7QOFtqhH3UxB7va5fFHfw6pCHzT7E1iM",
    ,"datetime"#: "2019-09-01 00:45:50 -0500", DONE
    ,"sender"#: "helpdesk@staefa.com.br", DONE
    ,"recipients"#: [ "brpjbissup@wal-mart.com" ], 
    ,"subject"#: null,
    ,"attachments"#: null,
    ,"rule"#: "norcpts",
    ,"action"#: "discard",
    ,"attach_cnt"#: "0",
    ,"rcpt_cnt"#: "1",
    ,"email_size"#: "0",
    ,"log_file"#: "201908.gz",
    ,"log_key_id"#: "2uqnqbb41y"]
          ]

### generate datetime

In [ ]:
#s_date = my_db.gen_data_series(DATA_SIZE, data_type='date') + 'T' + my_db.gen_data_series(DATA_SIZE, data_type='time') + 'Z'
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

In [ ]:
# RANDOM_DATES = True
# s_date = None
# if RANDOM_DATES:
    # this line give dates in a random range
s_date = pd.Series(random_dates(pd.to_datetime(START_DATE),pd.to_datetime(END_DATE),DATA_SIZE))
# else:
#     # this line limits dates and times to a single day
#     s_date = pd.Series(random_dates(pd.to_datetime('2019-09-14'),pd.to_datetime('2019-09-15'),DATA_SIZE))

In [ ]:
s_date = s_date.dt.tz_localize('UTC')

In [ ]:
s_date.dt.day.value_counts()

### Subject of email

In [ ]:
s_subjects = pd.Series([fake.text(max_nb_chars=64) for i in tqdm(range(DATA_SIZE))])

### GUID

In [ ]:
s_guids = pd.Series([randomString(32) for i in tqdm(range(DATA_SIZE))])

### Senders

In [ ]:
s_senders = pd.Series([random.choice(df_employees.email) for i in tqdm(range(DATA_SIZE))])

### Recipients

In [ ]:
def get_skewed_proba():
    pers = np.arange(1,32,1)

    # Make each of the last 41 elements 5x more likely
    prob = [10.0]*4 + [1.0]*(len(pers)-4)

    # Normalising to 1.0
    prob /= np.sum(prob)

    num_recips = np.random.choice(pers, 1, p=prob)
    return num_recips

In [ ]:
s_recipients = pd.Series([[random.choice(df_employees.email) for i in range(int(get_skewed_proba())) ] for j in tqdm(range(DATA_SIZE))])

### recip count

In [ ]:
s_rcpt_cnt = s_recipients.apply(lambda x: len(x))

### attachements

In [ ]:
def getAttachments():
    dirs = []
    for i in range(random.randint(0, 5)):
        dir = randomDir() 
        if dir:
            dirs.append(dir)
    return dirs

s_attachments = pd.Series([getAttachments() for i in tqdm(range(DATA_SIZE))])

### attachement count

In [ ]:
s_attach_cnt = s_attachments.apply(lambda x: len(x))

### rules

In [ ]:
rules = [fake.text(10).replace('.','').upper().split()[0] for i in range(100)]

In [ ]:
s_rule = pd.Series([random.choice(rules) for i in tqdm(range(DATA_SIZE))])

### actions

In [ ]:
actions = [fake.text(20).replace('.','').replace(' ','_').upper() for i in range(100)]

In [ ]:
s_action = pd.Series([random.choice(actions) for i in tqdm(range(DATA_SIZE))])

### log key id

In [ ]:
s_log_key_ids = pd.Series([randomString(8) for i in tqdm(range(int(DATA_SIZE/10)))])

### log file

In [ ]:
log_files = [f'20190{i}.gz' for i in range(1,10)]

In [ ]:
s_log_files = pd.Series([random.choice(log_files) for i in tqdm(range(DATA_SIZE))])

### email size

In [ ]:
s_email_size = pd.Series(np.random.randint(1024*1024*25, size=DATA_SIZE))

# Combined

In [ ]:
df_combined = pd.concat([s_guids, s_date, s_senders, s_recipients, s_subjects, s_attachments, s_rule, s_action, s_attach_cnt, s_rcpt_cnt, s_email_size, s_log_files, s_log_key_ids], axis=1)

In [ ]:
df_combined.columns = column_names

In [ ]:
df_combined.head()

In [ ]:
df_combined = df_combined.sort_values(['datetime'])

In [ ]:
# if not os.path.exists('synthetic_data'):
#     os.mkdir("synthetic_data")

In [ ]:
df_tmp = None
for date in tqdm(df_combined.datetime.dt.date.unique()):
    df_tmp = df_combined[df_combined.datetime.dt.date == date]
    df_tmp.to_csv(f'../airflow/data/{date}.csv', index=False)

## Explore

In [ ]:
from glob import glob

In [ ]:
files = glob('synthetic_data/*')

In [ ]:
df_files = pd.DataFrame(files, columns=['file_name'])

In [ ]:
df_files['time_created'] = pd.to_datetime(df_files.file_name.apply(os.path.getatime),unit='s')

In [ ]:
df_files['time_modified'] = pd.to_datetime(df_files.file_name.apply(os.path.getmtime),unit='s')

In [ ]:
df_files.time_created.dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
df_files[df_files.time_created.dt.strftime('%Y-%m-%d %H:%M') != df_files.time_modified.dt.strftime('%Y-%m-%d %H:%M')]

In [ ]:
os.path.getctime(files[0])

In [ ]:
os.path.getmtime(files[0])

In [ ]:
os.path.getatime(files[0])

In [ ]:
files[0]

In [ ]:
os.stat(files[0])

## Employees to JSON

In [ ]:
import json
import csv

In [ ]:
df_employees = pd.read_csv('employees.csv', index_col=0)

In [ ]:
df_employees.to_csv('../airflow/data/employees.csv', index=None, header=True, quoting=csv.QUOTE_ALL)

In [ ]:
!head ../airflow/data/employees.csv

In [ ]:
print(json.dumps(json.loads(df_employees.head().to_json(orient='records')), indent=2))

In [ ]:
df_employees.to_json('/encrypted_vol/jupyter_data/airflow/data/employees.json', orient='records')

In [ ]:
df_employees.address = df_employees.address.str.replace('\n',', ')

In [ ]:
with open('employees.json', 'wt') as f:
    for i, row in df_employees.iterrows():
        f.write(row.to_json()+ '\n')

In [ ]:
!head employees.json

In [ ]:
df_employees.email.count()